<strong>Дано числовое уравнение, т.е. уравнение в одной вещественной неизвестной.
    
A. Преобразовать уравнение к виду, пригодному для применения принципа
сжимающих операторов.

B. Методом простых итераций найти приближенное решение с точностью $10^{-5}$, используя априорную и апостериорную оценки числа итераций.</strong>

$x + \frac{x}{x^{2} + 1} + \sin{\left (\frac{x}{2} \right )} = 5$

<i>A. Преобразуем уравнение к виду, пригодному для применения принципа сжимающих
операторов.</i>

Обозначим $\varphi(x) = x + \frac{x}{x^{2} + 1} + \sin{\left (\frac{x}{2} \right )} - 5$, причем $\varphi(x) = 0$. 

Вычислим $\varphi^{'}(x)$:

In [1]:
import sympy as sp
from IPython.display import display, Latex

sp.init_printing(use_latex='mathjax')

x = sp.Symbol('x')
varphi = (x + sp.sin(x/2) + x/(1 + x**2) - 5) 

diffed_varphi = sp.diff(varphi, x)
display(diffed_varphi) 

                 ⎛x⎞             
        2     cos⎜─⎟             
     2⋅x         ⎝2⎠         1   
- ───────── + ────── + 1 + ──────
          2     2           2    
  ⎛ 2    ⎞                 x  + 1
  ⎝x  + 1⎠                       

Оценим функцию сверху:

$$\frac{1}{2} \cos{\left (\frac{x}{2} \right )} \leq \frac{1}{2}$$
$$\frac{1}{x^{2} + 1} \leq 1$$
$$1 - \frac{2x^{2}}{(x^{2} + 1)^{2}} \leq 1$$


Найдем минимум функции, используя алгоритм, реализующий метод Нелдера — Мида:

In [2]:
from scipy.optimize import fmin

research = lambda t: diffed_varphi.subs(x, t)

# Возьмем произвольну точку -5
xmin0 = fmin(research, -5)

Optimization terminated successfully.
         Current function value: 0.476320
         Iterations: 16
         Function evaluations: 32


Таким образом, 
$$0.476320 \leq \varphi^{'}(x) \leq \frac{5}{2}$$
$$0 < \nu \leq \varphi^{'}(x) \leq \mu$$
$$ \nu = 0.476320, \mu = \frac{5}{2}$$


Преобразуем уравнение $\varphi(x) = 0$ к виду $x - \frac{1}{\mu}\varphi(x) = x$:
$$x - \frac{2}{5}(x + \frac{x}{x^{2} + 1} + \sin{\left (\frac{x}{2} \right )} - 5) = x$$

Рассмотрим оператор 
$$Ф: R \to R, Ф[x] = x - \frac{2}{5}(x + \frac{x}{x^{2} + 1} + \sin{\left (\frac{x}{2} \right )} - 5)$$

Это уравнение имеет вид $Ф[x] = x$, его решение – неподвижная точка оператора $Ф$. Оператор $Ф$ является сжимающим с коэффициентом сжатия
$$\alpha = 1 - \frac{\nu}{\mu}$$

In [3]:
nu = 0.476320
mu = sp.Rational(5, 2)

alpha = 1 - nu/mu
epsilon = 10**(-5)

display(alpha)

0.809472000000000

Таким образом, нашли $\alpha$.

<i>B. Используя метод простых итераций, найдем приближенное решение уравнения с точностью $\varepsilon = 10^5$. Процесс вычислений организуем с помощью априорной и апостериорной оценок числа итераций.</i>

Для произвольного начального приближения $x_0$ последовательность итераций задается рекуррентной формулой $x_n = Ф[x_{n - 1}]$.В данном случае:

$$x_n = x_{n - 1} - \frac{2}{5}(x_{n - 1} + \frac{x_{n - 1}}{x_{n - 1}^{2} + 1} + \sin{\left (\frac{x_{n - 1}}{2} \right)} - 5)$$


Выберем произвольным образом начальное приближение, например $x_0 = 6$, и вычисляем
первую итерацию $x_1$:

In [4]:
import math

def dist(x1, x0):
    return math.fabs(x1 - x0)

def is_enough(alp, x1, x0, eps): 
    return (alp/(1 - alp))*dist(x1, x0) <= eps

def apriori_weight(alp, eps, x0, x1):
    return round(math.log(eps*(1 - alp)/dist(x0, x1), alp) + 1)

In [5]:
f = x - 0.4*varphi

x_0 = 6
x_1 = f.subs(x, x_0)
apriori = apriori_weight(alpha, epsilon, x_0, x_1)

display(Latex("Найдем априорную оценку числа итераций $N_{apr}$:"))
display(apriori)
display(Latex("Следовательно, для вычисления приближенного решения " + 
              "с требуемой точностью понадобится не более, чем " + str(apriori) + " итераций"))

<IPython.core.display.Latex object>

60

<IPython.core.display.Latex object>

In [6]:
x_curr = x_0
i = 0
while True:
    i += 1
    x_prev = x_curr
    x_curr = sp.N(f.subs(x, x_prev))
    if is_enough(alpha, x_prev, x_curr, epsilon):
        break
        
display(Latex("В данном случае понадобилось всего " + str(i) + " итераций"))
display(Latex("Получено приближенное решение x_" + str(i) + " = " + str(x_curr)))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

Как видно, априорная оценка числа итераций сильно завышена. 